<a href="https://colab.research.google.com/github/justinf34/SENG550-Project/blob/main/DataCleaningAndPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up Colab Environment


#### Mounting google drive to this colab notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import os

os.chdir('drive/MyDrive/Colab Notebooks/SENG550/dataset/') # Chaging to dataset dir
print(os.listdir())

['LeagueofLegends.csv', 'gold.csv', '_columns.csv', 'bans.csv', 'kills.csv', 'monsters.csv', 'structures.csv', 'matchinfo.csv', 'Reference-Dataset', 'model_features']


#### Downloading and Setting up Spark

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 47.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=c981f3022f75370cb791aa10d6dde8efe6fec4d1b57412496229a8af19d6e6b5
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
# Setting up spark
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("SENG550").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [ ]:
sc.stop()

In [ ]:
# Setting up Spark context using SparkSession
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark.stop()

## Cleaning Data & Preprocessing Data


Loading the data and creating an RDD.

In [ ]:
filename = 'LeagueofLegends.csv'
fileDir = os.path.join(os.getcwd(),filename)

games_df = spark.read.csv(fileDir,inferSchema=True,header=True)
games_df.createOrReplaceTempView("games")
games_df.printSchema()

root
 |-- League: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Season: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- blueTeamTag: string (nullable = true)
 |-- bResult: integer (nullable = true)
 |-- rResult: integer (nullable = true)
 |-- redTeamTag: string (nullable = true)
 |-- gamelength: integer (nullable = true)
 |-- golddiff: string (nullable = true)
 |-- goldblue: string (nullable = true)
 |-- bKills: string (nullable = true)
 |-- bTowers: string (nullable = true)
 |-- bInhibs: string (nullable = true)
 |-- bDragons: string (nullable = true)
 |-- bBarons: string (nullable = true)
 |-- bHeralds: string (nullable = true)
 |-- goldred: string (nullable = true)
 |-- rKills: string (nullable = true)
 |-- rTowers: string (nullable = true)
 |-- rInhibs: string (nullable = true)
 |-- rDragons: string (nullable = true)
 |-- rBarons: string (nullable = true)
 |-- rHeralds: string (nullable = true)
 |-- blueTop: string (nullable = true)
 |-- blueT

In [ ]:
get_cols_query = 'SELECT bResult, rResult, golddiff, bKills, \
               bTowers, bInhibs, bDragons, bBarons, \
               bHeralds, goldred, rKills, rTowers, \
               rInhibs, rDragons, rBarons, rHeralds \
               FROM games'
basic_team_stats_df = spark.sql(get_cols_query)
basic_team_stats_df.printSchema()
basic_team_stats_df.show()

root
 |-- bResult: integer (nullable = true)
 |-- rResult: integer (nullable = true)
 |-- golddiff: string (nullable = true)
 |-- bKills: string (nullable = true)
 |-- bTowers: string (nullable = true)
 |-- bInhibs: string (nullable = true)
 |-- bDragons: string (nullable = true)
 |-- bBarons: string (nullable = true)
 |-- bHeralds: string (nullable = true)
 |-- goldred: string (nullable = true)
 |-- rKills: string (nullable = true)
 |-- rTowers: string (nullable = true)
 |-- rInhibs: string (nullable = true)
 |-- rDragons: string (nullable = true)
 |-- rBarons: string (nullable = true)
 |-- rHeralds: string (nullable = true)

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|bResult|rResult|            golddiff|              bKills|             bTowe

#### Part(1a): Winner Column

For this step, we are adding a new colum `winner` that indicates which team won the match(record). `1` will indicate as the blue being the winner and `0` for the red team. This new schema will be added to a new dataframe.

In [ ]:
import pyspark.sql.functions as f
import pyspark.sql.types as types

def calcWinner(blue , red):
  print(red, blue)
  if blue == 1:
    return 1
  else:
    return 0

udf_calcWinner = f.udf(calcWinner, types.IntegerType())

new_df = basic_team_stats_df.withColumn("winner", udf_calcWinner(f.col('bResult'), f.col('rResult')))
new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|
+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+
|      1|      0|[0, 0, -14, -65, ...|[[10.82, 'C9 Hai'...|[[27.542, 'MID_LA...|[[36.686, 'MID_LA...|    [[37.267, 

#### Part(1b): Final Gold diff Column

For this step we are calculating the final gold difference in the game. The `golddiff` column in the original contains a list of gold difference at different times of the match and is represented as a string. The gold differences is calculated by subtractice red gold to blue gold. To get the final gold difference, we just need to extract the last gold difference in the list.

In [ ]:
GGdef finalGoldDiff(golddiff):
  gDiff = golddiff
  gDiff = gDiff.rstrip("]").lstrip("[")
  gDiff = list(map(int, gDiff.split(",")))
  
  return gDiff[len(gDiff) - 1]

udf_finalGoldDiff = f.udf(finalGoldDiff, types.StringType())

new_df = new_df.withColumn("final_golddiff", udf_finalGoldDiff(f.col('golddiff')))
new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|final_golddiff|
+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+
|      1|      0|[0, 0, -14, -65, ...|[[10.82, 'C9 Hai'...|[[27.542, '

#### Part(1c): Kill Count Columns

In this step, we are determining how many kills each team got in each game.

In [ ]:
import ast

def countKills(kills):
  return len(ast.literal_eval(kills))

udf_countKills = f.udf(countKills, types.StringType())

In [ ]:
new_df = new_df.withColumn("bKillCount", udf_countKills(f.col('bKills')))\
         .withColumn("rKillCount", udf_countKills(f.col('rKills')))

new_df.show()

#### Part(1d): First Blood Column

In this step, we are determining which team got the first kill of the game

In [ ]:
def firstKill(bKills, rKills):
  blue_fk = ast.literal_eval(bKills)
  red_fk = ast.literal_eval(rKills)

  if(len(blue_fk) == 0 and len(red_fk) == 0): return -1
  if(len(blue_fk) == 0): return 0
  if(len(red_fk) == 0): return 1

  return 1 if blue_fk[0][0] < red_fk[0][0] else 0

udf_firstKill = f.udf(firstKill, types.StringType())

In [ ]:
new_df = new_df.withColumn("first_blood", udf_firstKill(f.col('bKills'), f.col('rKills')))
new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+-----------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|first_blood|
+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+-----------+
|      1|      0|[0, 0, -14, -65, ...|[[10.82, 'C9 Hai'...|[[27.542, 'MID_LA...

#### Part(1e): Tower, Inhib, Dragon, Baron, Herald

For counting and calculating who got the other objective first, we can re use the same function as their formats are very similar.

In [ ]:
def firstObj(bObj, rObj):
  blue = ast.literal_eval(bObj)
  red = ast.literal_eval(rObj)

  blue = list(filter(None, blue))
  red = list(filter(None, red))

  if(len(blue) == 0 and len(red) == 0): return -1
  if(len(blue) == 0): return 0
  if(len(red) == 0): return 1

  return 1 if blue[0][0] < red[0][0] else 0

udf_firstObj = f.udf(firstObj, types.StringType());

In [ ]:
def countObj(obj):
  obj_lists = ast.literal_eval(obj)
  obj_lists = list(filter(None, obj_lists))
  return len(obj_lists)

udf_countObj = f.udf(countObj, types.StringType())

##### Tower
For this part, we are calcuating how many towers each team knocked down and which team got the first tower.

In [ ]:
new_df = new_df.withColumn('bTowerCount', udf_countObj(f.col('bTowers')))\
         .withColumn('rTowerCount', udf_countObj(f.col('rTowers')))\
         .withColumn('first_tower', udf_firstObj(f.col('bTowers'), f.col('rTowers')))

new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+----------+----------+-----------+-----------+-----------+-----------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|final_golddiff|bKillCount|rKillCount|first_blood|bTowerCount|rTowerCount|first_tower|
+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+----

##### Inhibitors

For this part, we are calculating how many inhibitors each team knocked down and which team knocked down the first inhib.

In [ ]:
new_df = new_df.withColumn('bInhibCount', udf_countObj(f.col('bInhibs')))\
         .withColumn('rInhibCount', udf_countObj(f.col('rInhibs')))\
         .withColumn('first_inhib', udf_firstObj(f.col('bInhibs'), f.col('rInhibs')))

new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|final_golddiff|bKillCount|rKillCount|first_blood|bTowerCount|rTowerCount|first_tower|bInhibCount|rInhibCount|first_inhib|
+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+----------------

##### Dragons

For this part, we are calculating how many dragons each team killed and which team got the first dragon.

In [ ]:
new_df = new_df.withColumn('bDragonCount', udf_countObj(f.col('bDragons')))\
         .withColumn('rDragonCount', udf_countObj(f.col('rDragons')))\
         .withColumn('first_dragon', udf_firstObj(f.col('bDragons'), f.col('rDragons')))

new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|final_golddiff|bKillCount|rKillCount|first_blood|bTowerCount|rTowerCount|first_tower|bInhibCount|rInhibCount|first_inhib|bDragonCount|rDragonCount|first_dragon|
+-------+-------+--------------------+--------------------+--------------------+----------

##### Barons

For this part, we are calculating how many barons each team killed and which team got the first baron.

In [ ]:
new_df = new_df.withColumn('bBaronCount', udf_countObj(f.col('bBarons')))\
         .withColumn('rBaronCount', udf_countObj(f.col('rBarons')))\
         .withColumn('first_baron', udf_firstObj(f.col('bBarons'), f.col('rBarons')))

new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+-----------+-----------+-----------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|final_golddiff|bKillCount|rKillCount|first_blood|bTowerCount|rTowerCount|first_tower|bInhibCount|rInhibCount|first_inhib|bDragonCount|rDragonCount|first_dragon|bBaronCount|rBaronCount|first_baron|
+-------+-------+-

##### Heralds

For this part, we are calculating how many heralds each team killed and which team got the first herald.

In [ ]:
new_df = new_df.withColumn('bHeraldCount', udf_countObj(f.col('bHeralds')))\
         .withColumn('rHeraldCount', udf_countObj(f.col('rHeralds')))\
         .withColumn('first_herald', udf_firstObj(f.col('bHeralds'), f.col('rHeralds')))

new_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+--------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+-----------+-----------+-----------+------------+------------+------------+
|bResult|rResult|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|final_golddiff|bKillCount|rKillCount|first_blood|bTowerCount|rTowerCount|first_tower|bInhibCount|rInhibCount|first_inhib|bDragonCount|rDragonCount|first_dragon|bBaronCount|rBar

#### Part(1f): Writing the data

After we got the necessary columns for our models, we are going to store a subste of `new_df` that only contains columns that our models need. This will make things easier later as we can interact with the dataset without treating them as dataframes.

In [ ]:
model_df = new_df.select('winner', 'final_golddiff', 'bKillCount', 'rKillCount', \
                         'first_blood', 'bTowerCount', 'rTowerCount', 'first_tower',\
                         'bInhibCount', 'rInhibCount', 'first_inhib', 'bDragonCount', \
                         'rDragonCount', 'first_dragon', 'bBaronCount', 'rBaronCount', \
                         'first_baron', 'bHeraldCount', 'rHeraldCount', 'first_herald')
model_df.printSchema()

root
 |-- winner: integer (nullable = true)
 |-- final_golddiff: string (nullable = true)
 |-- bKillCount: string (nullable = true)
 |-- rKillCount: string (nullable = true)
 |-- first_blood: string (nullable = true)
 |-- bTowerCount: string (nullable = true)
 |-- rTowerCount: string (nullable = true)
 |-- first_tower: string (nullable = true)
 |-- bInhibCount: string (nullable = true)
 |-- rInhibCount: string (nullable = true)
 |-- first_inhib: string (nullable = true)
 |-- bDragonCount: string (nullable = true)
 |-- rDragonCount: string (nullable = true)
 |-- first_dragon: string (nullable = true)
 |-- bBaronCount: string (nullable = true)
 |-- rBaronCount: string (nullable = true)
 |-- first_baron: string (nullable = true)
 |-- bHeraldCount: string (nullable = true)
 |-- rHeraldCount: string (nullable = true)
 |-- first_herald: string (nullable = true)



In [ ]:
file_dir = os.path.join(os.getcwd(), 'model_features')

model_df.write.format('csv').option('header', False)\
.mode('overwrite').save(file_dir)

## Mid-game Stats Processing

In this section, we are gonna be doing the same thing a **Part 1**, but this time, we are only extracting mid-game stats ranther than the end game. We feel like a model that can predict which team will win looking at the mid-game will provide more value than model that uses end-game stats.

In [ ]:
get_cols_query2 = 'SELECT bResult, rResult, gamelength, golddiff, bKills, \
               bTowers, bInhibs, bDragons, bBarons, \
               bHeralds, goldred, rKills, rTowers, \
               rInhibs, rDragons, rBarons, rHeralds \
               FROM games'
match_df = spark.sql(get_cols_query2)

### Winner Calculation

It will use the same function as **Part 1**.

In [ ]:
mid_df = match_df.withColumn("winner", udf_calcWinner(f.col('bResult'), f.col('rResult')))
mid_df.show()

+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+
|bResult|rResult|gamelength|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|
+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+
|      1|      0|        40|[0, 0, -14, -65, ...|[[10.82, 'C9 Hai'...|[[27.542, 'M

### Other Columns
For the other columns the function will be very similar to **Part 1** but this time, it will take into account the game duration and only use the values that occur before the half-way point of the game.

In [ ]:
mid_df = mid_df.withColumn("first_blood", udf_firstKill(f.col('bKills'), f.col('rKills')))
mid_df.show()

+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+-----------+-----------+-----------+-----------+
|bResult|rResult|gamelength|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|mid_golddiff|bKillCount|rKillCount|bTowerCount|rTowerCount|first_tower|bInhibCount|rInhibCount|first_inhib|bDragonCount|rDragonCount|first_dragon|bBaronCount|rBaronCount|first_baron|first_blood|


In [ ]:
import ast

count = 0
test = mid_df.first()
arr = ast.literal_eval(test["bKills"])
mid_time = float(test["gamelength"]) / 2
print(mid_time)
for k in arr:
  if k[0] <= mid_time: 
    print(k[0])
    count += 1

# print(type(test["gamelength"]))

20.0
10.82
16.286
18.733
18.88


In [ ]:
import ast

def midGoldDiff(golddiff, gamelength):
  gDiff = ast.literal_eval(golddiff)
  gLength = int(gamelength)
  mid = -(-gLength//2)
  
  return gDiff[mid - 1]

udf_midGoldDiff = f.udf(midGoldDiff, types.StringType())


mid_df = mid_df.withColumn("mid_golddiff", udf_midGoldDiff(f.col('golddiff'), f.col('gamelength')))
mid_df.show()

+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+------------+
|bResult|rResult|gamelength|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|mid_golddiff|
+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+------------+
|      1|      0|        40|[0, 0, -14, -65

In [ ]:
def midCountKills(kills, gamelength):
  kills_arr = ast.literal_eval(kills)
  mid_time = float(gamelength) / 2

  count = 0
  for kill in kills_arr:
    if kill[0] <= mid_time: 
      count += 1
  
  return count

udf_midCountKills = f.udf(midCountKills, types.StringType())


mid_df = mid_df.withColumn("bKillCount", udf_midCountKills(f.col('bKills'),f.col('gamelength')))\
         .withColumn("rKillCount", udf_midCountKills(f.col('rKills'), f.col('gamelength')))

mid_df.show()

+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+------------+----------+----------+
|bResult|rResult|gamelength|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|mid_golddiff|bKillCount|rKillCount|
+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+------------+

In [ ]:
def midCountObj(obj, gamelength):
  obj_lists = ast.literal_eval(obj)
  obj_lists = list(filter(None, obj_lists))
  mid_time = float(gamelength) / 2

  count = 0
  for obj in obj_lists:
    if obj[0] <= mid_time:
      count += 1

  return count 

udf_midCountObj = f.udf(midCountObj, types.StringType())    

In [ ]:
mid_df = mid_df.withColumn('bTowerCount', udf_midCountObj(f.col('bTowers'),f.col('gamelength')))\
         .withColumn('rTowerCount', udf_midCountObj(f.col('rTowers'), f.col('gamelength')))\
         .withColumn('first_tower', udf_firstObj(f.col('bTowers'), f.col('rTowers')))\
         .withColumn('bInhibCount', udf_midCountObj(f.col('bInhibs'),f.col('gamelength')))\
         .withColumn('rInhibCount', udf_midCountObj(f.col('rInhibs'),f.col('gamelength')))\
         .withColumn('first_inhib', udf_firstObj(f.col('bInhibs'), f.col('rInhibs')))\
         .withColumn('bDragonCount', udf_midCountObj(f.col('bDragons'),f.col('gamelength')))\
         .withColumn('rDragonCount', udf_midCountObj(f.col('rDragons'),f.col('gamelength')))\
         .withColumn('first_dragon', udf_firstObj(f.col('bDragons'), f.col('rDragons')))\
         .withColumn('bBaronCount', udf_midCountObj(f.col('bBarons'),f.col('gamelength')))\
         .withColumn('rBaronCount', udf_midCountObj(f.col('rBarons'),f.col('gamelength')))\
         .withColumn('first_baron', udf_firstObj(f.col('bBarons'), f.col('rBarons')))


mid_df.show()    

+-------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------+------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+-----------+-----------+-----------+
|bResult|rResult|gamelength|            golddiff|              bKills|             bTowers|             bInhibs|            bDragons|             bBarons|bHeralds|             goldred|              rKills|             rTowers|             rInhibs|            rDragons|             rBarons|rHeralds|winner|mid_golddiff|bKillCount|rKillCount|bTowerCount|rTowerCount|first_tower|bInhibCount|rInhibCount|first_inhib|bDragonCount|rDragonCount|first_dragon|bBaronCount|rBaronCount|first_baron|
+-------+-------+-------

In [ ]:
mid_model_df = mid_df.select('winner', 'mid_golddiff', 'bKillCount', 'rKillCount', \
                         'first_blood', 'bTowerCount', 'rTowerCount', 'first_tower',\
                         'bInhibCount', 'rInhibCount', 'first_inhib', 'bDragonCount', \
                         'rDragonCount', 'first_dragon', 'bBaronCount', 'rBaronCount', \
                         'first_baron')
mid_model_df.printSchema()

root
 |-- winner: integer (nullable = true)
 |-- mid_golddiff: string (nullable = true)
 |-- bKillCount: string (nullable = true)
 |-- rKillCount: string (nullable = true)
 |-- first_blood: string (nullable = true)
 |-- bTowerCount: string (nullable = true)
 |-- rTowerCount: string (nullable = true)
 |-- first_tower: string (nullable = true)
 |-- bInhibCount: string (nullable = true)
 |-- rInhibCount: string (nullable = true)
 |-- first_inhib: string (nullable = true)
 |-- bDragonCount: string (nullable = true)
 |-- rDragonCount: string (nullable = true)
 |-- first_dragon: string (nullable = true)
 |-- bBaronCount: string (nullable = true)
 |-- rBaronCount: string (nullable = true)
 |-- first_baron: string (nullable = true)



In [ ]:
file_dir = os.path.join(os.getcwd(), 'mid_model_features')

mid_model_df.write.format('csv').option('header', False)\
.mode('overwrite').save(file_dir)